<a href="https://colab.research.google.com/github/shruthad/Fake-news-detection/blob/main/Fake_news_detection(tokenizer_rnn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
import re 
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import seaborn as sns 
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:

real_df=pd.read_csv('True.csv')
fake_df=pd.read_csv('Fake.csv')

In [ ]:
fake_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [ ]:
real_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [ ]:
fake_df.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [ ]:
real_df.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [ ]:
fake_df.drop(['date', 'subject'], axis=1, inplace=True)
real_df.drop(['date', 'subject'], axis=1, inplace=True)

In [ ]:
fake_df['class'] = 0 
real_df['class'] = 1

In [ ]:
news_df = pd.concat([fake_df, real_df], ignore_index=True, sort=False)
news_df

,title,text,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0
...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [ ]:
news_df['text'] = news_df['title'] + news_df['text']
news_df.drop('title', axis=1, inplace=True)

In [ ]:
features = news_df['text']
targets = news_df['class']

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.20, random_state=18)

In [ ]:
def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

In [ ]:
# tokenize the text into vectors 
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train[0]

array([  98,    2,  207, 7153, 5740,  256,  492,  132,  284, 1479,  841,
       1729,  321,  574, 2547,   35,    7,  244,   11,    4, 1560,  114,
        284, 8580, 6626, 5500,  298,  948, 1303, 3574, 1933,   18,   51,
          3,    1,  197,    3,    1,  155,    9,  227, 6626,    7,  362,
          6,   53,  953, 1406, 3810,   31,  841, 3077,  489,    2,   53,
          5,    1,  627,  284, 4233, 1936,   10,    6,  178,  436,  813,
         49,  284, 4079, 2377,   17, 3332,    6,    2, 1424, 1936, 6626,
        267,    2,    1, 6585,    6,  586,    9,  438,    2,  212,  691,
         34, 1659,  298,  948,    7,  428,  244,   18,    4,  431,   32,
        729,  140,    2,    1, 4218, 1696,    3,  179,  815,    2,    1,
        132,    1,  840,   17, 6181,  599,    1,  161,   17, 8444,    2,
        321,   10,  284, 1936,    1,  161,   17, 7533, 6626,   15,  379,
          7,  715,  258,    3,  201,    1,  258,    8,  841,   21,  313,
       5900,  727,   36, 1127, 1703,    5,  244,  3

In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=256)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=256)

In [ ]:
#embedding_vector_features=40
model=Sequential()
model.add(Embedding(max_vocab, 32))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train,y_train,validation_split=0.2,epochs=10,batch_size=64)

Epoch 1/10
449/449 [==============================] - 42s 21ms/step - loss: 0.4692 - accuracy: 0.7904 - val_loss: 0.1434 - val_accuracy: 0.9603
Epoch 2/10
449/449 [==============================] - 9s 20ms/step - loss: 0.2788 - accuracy: 0.8689 - val_loss: 0.1110 - val_accuracy: 0.9687
Epoch 3/10
449/449 [==============================] - 9s 21ms/step - loss: 0.1194 - accuracy: 0.9626 - val_loss: 0.0998 - val_accuracy: 0.9690
Epoch 4/10
449/449 [==============================] - 10s 21ms/step - loss: 0.0896 - accuracy: 0.9738 - val_loss: 0.0986 - val_accuracy: 0.9724
Epoch 5/10
449/449 [==============================] - 10s 21ms/step - loss: 0.0756 - accuracy: 0.9805 - val_loss: 0.1269 - val_accuracy: 0.9667
Epoch 6/10
449/449 [==============================] - 10s 22ms/step - loss: 0.1669 - accuracy: 0.9527 - val_loss: 0.2299 - val_accuracy: 0.8934
Epoch 7/10
449/449 [==============================] - 10s 21ms/step - loss: 0.2191 - accuracy: 0.9024 - val_loss: 0.1160 - val_accuracy: 0

In [ ]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[4368,  294],
       [ 126, 4192]])

In [ ]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9532293986636972